In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/commonlit-evaluate-student-summaries/sample_submission.csv
/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv
/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv
/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv
/kaggle/input/commonlit-evaluate-student-summaries/prompts_test.csv


In [2]:
!nvidia-smi -L

/bin/bash: nvidia-smi: command not found


In [3]:
def print_tuple(obs):
    print("\033[1mprompt_question: \033[0m", obs.prompt_question)
    print()
    print("\033[1mprompt_title: \033[0m", obs.prompt_title)
    print()
    print("\033[1mprompt_text: \033[0m")
    print(obs.prompt_text)
    print()
    print("\033[1msummary: \033[0m")
    print(obs.text)
    print()
    print("\033[1mWording: \033[0m")
    print(obs.wording)
    print()
    print("\033[1mContent: \033[0m")
    print(obs.content)
    

In [4]:
#Filepath
FILE_PATH='/kaggle/input/commonlit-evaluate-student-summaries'
#train set
pmt_train=pd.read_csv(os.path.join(FILE_PATH,'prompts_train.csv'))
sum_train=pd.read_csv(os.path.join(FILE_PATH,'summaries_train.csv'))
df_train=pmt_train.merge(sum_train,how='inner',on='prompt_id')

In [5]:
#df_train
tuple1=df_train.iloc[0]
print_tuple(tuple1)

prompt_question:  Summarize at least 3 elements of an ideal tragedy, as described by Aristotle.

prompt_title:  On Tragedy

prompt_text: 
Chapter 13 
As the sequel to what has already been said, we must proceed to consider what the poet should aim at, and what he should avoid, in constructing his plots; and by what means the specific effect of Tragedy will be produced. 
A perfect tragedy should, as we have seen, be arranged not on the simple but on the complex plan. It should, moreover, imitate actions which excite pity and fear, this being the distinctive mark of tragic imitation. It follows plainly, in the first place, that the change of fortune presented must not be the spectacle of a virtuous man brought from prosperity to adversity: for this moves neither pity nor fear; it merely shocks us. Nor, again, that of a bad man passing from adversity to prosperity: for nothing can be more alien to the spirit of Tragedy; it possesses no single tragic quality; it neither satisfies the moral

In [6]:
#we actually dont want student_id or prompt_id for training hence dropping those.
df_train.drop(['student_id','prompt_id'],axis=1,inplace=True)

### Tokenizing and preprocessing

We will be using tokenizers like Bert and RoBERTa

In [7]:
import numpy as np
from string import Template
import tqdm
import torch
from sklearn.decomposition import PCA
from transformers import (BertTokenizer, BertModel , 
                         T5Tokenizer, T5ForConditionalGeneration , 
                         AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer , 
                         AutoModel)
#tokenizer associated with bert-base-cased will be downloaded.
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [8]:
#Tokenizing Endymion by john keats
bert_tokenizer("A thing of beauty is a joy for ever:Its loveliness increases; it will neverPass into nothingness",
    return_tensors = "pt")

{'input_ids': tensor([[  101,   138,  1645,  1104,  5295,  1110,   170,  8730,  1111,  1518,
           131,  2098,  1567, 14951,  6986,   132,  1122,  1209,  1309,  2101,
         11192,  1154,  1720,  1757,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]])}

In [9]:
def tokenize(value , tokenizer, index = "input_ids"):
    
    if len(value) > tokenizer.model_max_length:value = value[:tokenizer.model_max_length]
        
    tokens = tokenizer(value , return_tensors = "pt")

    return tokens[index]

In [10]:
#Column wise tokenization
for columns in ["text"]:
    for index in tqdm.tqdm(range(df_train.shape[0]) , total = df_train.shape[0]):
        df_train[columns][index] = tokenize(df_train[columns][index] , tokenizer = bert_tokenizer , index = "input_ids")

  0%|          | 0/7165 [00:00<?, ?it/s]/tmp/ipykernel_20/3835569631.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[columns][index] = tokenize(df_train[columns][index] , tokenizer = bert_tokenizer , index = "input_ids")
100%|██████████| 7165/7165 [00:13<00:00, 517.00it/s]


In [11]:
df_train['text'][0]
#The test is vectorized but not embedded

tensor([[  101,   122,  5290,  1104,  1126,  7891, 12343,  1110,  1115,  1122,
          1431,  1129,  4768,  1113,   170,  2703,  2197,   119,  2543,  5290,
          1104,  1126,  7891, 12343,  1110,  1115,  1122,  1431,  1178,  1138,
          1141,  1514,  2486,   119,  1109,  1314,  5290,  1104,  1126,  7891,
         12343,  1110,  1115,  1122,  1431,  1138,   170,  2702, 12473,  4928,
          1105,  1126,  3714,  5855, 12788, 24658,  1111,  1241,  1363,  1105,
          2213,   119,   102]])

In [12]:
from sklearn.model_selection import train_test_split 
X = df_train['text'].tolist()
y = df_train[['wording', 'content']].values
X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.2, random_state=42)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
y_eval= tf.convert_to_tensor(y_test, dtype=tf.float32)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:5                                                                                    │
│                                                                                                  │
│   2 X = df_train['text'].tolist()                                                                │
│   3 y = df_train[['wording', 'content']].values                                                  │
│   4 X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.2, random_state=42     │
│ ❱ 5 y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)                                    │
│   6 y_eval= tf.convert_to_tensor(y_test, dtype=tf.float32)                                       │
│   7                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'tf' is not defined

In [ ]:
import tensorflow as tf
#Creating a model and doing predictions
tf.random.set_seed(42)
#model creation using sequential api
model01=tf.keras.Sequential([
    tf.keras.layers.Input(shape=[1,]),
    tf.keras.layers.Dense(64,activation="relu"),
    tf.keras.layers.Dense(1)
])
model01.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [ ]:
model01.fit(X_train, y_train, epochs=5, validation_data=(X_eval, y_eval))

In [ ]:
os.makedirs("/kaggle/working/Bert Based")
df_train.to_csv("/kaggle/working/Bert Based/Train Embeds")